In [1]:
!pip install swiftspec

In [2]:
import fsspec
async def get_client(**kwargs):
    import aiohttp
    import aiohttp_retry
    retry_options = aiohttp_retry.ExponentialRetry(
            attempts=3,
            exceptions={OSError, aiohttp.ServerDisconnectedError})
    retry_client = aiohttp_retry.RetryClient(raise_for_status=False, retry_options=retry_options)
    return retry_client

In [3]:
filepath="https://object-store.cloud.muni.cz/swift/v1/testfred"
filepath="https://object-store.cloud.muni.cz/swift/v1/foss4g-data/test/userdata.txt"

with fsspec.open(filepath, "r") as f:
    print(f.read())

#cloud-config
password: pangeo
chpasswd: { expire: False }
ssh_pwauth: True



In [4]:
filepath="https://object-store.cloud.muni.cz/swift/v1/testfred/"
with fsspec.open(filepath, "r") as f:
    print(f.read())

t_1.nc


In [5]:
import xarray as xr

In [6]:
!wget https://object-store.cloud.muni.cz/swift/v1/testfred/t_1.nc
ds = xr.open_dataset('./t_1.nc',engine='h5netcdf')
ds

--2022-07-30 07:51:22--  https://object-store.cloud.muni.cz/swift/v1/testfred/t_1.nc
Resolving object-store.cloud.muni.cz (object-store.cloud.muni.cz)... 147.251.62.11
Connecting to object-store.cloud.muni.cz (object-store.cloud.muni.cz)|147.251.62.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37954 (37K) [application/x-netcdf]
Saving to: ‘t_1.nc.3’

t_1.nc.3            100%[===================>]  37.06K  --.-KB/s    in 0.1s    

2022-07-30 07:51:23 (307 KB/s) - ‘t_1.nc.3’ saved [37954/37954]



<xarray.Dataset>
Dimensions:  (time: 4, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-01T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    history:      Thu Jul 28 19:01:29 2022: ncks --mk_rec_dmn time files/test...
    NCO:          netCDF Operators version 5.1.0 (Homepage = http://nco.sf.ne...

## Here we use 'https' protocol, and we can load netcdf file to xarray from the data we placed in cesnet's swift. 
- How can we set to use 'swift' protocol instead of 'https'?


In [7]:
%%time
f = fsspec.open('https://object-store.cloud.muni.cz/swift/v1/testfred/t_1.nc') 

CPU times: user 159 µs, sys: 28 µs, total: 187 µs
Wall time: 191 µs


In [8]:
%%time
ds = xr.open_dataset(f.open(), engine='h5netcdf')
ds

CPU times: user 42.3 ms, sys: 2.38 ms, total: 44.7 ms
Wall time: 427 ms


<xarray.Dataset>
Dimensions:  (time: 4, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-01T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    history:      Thu Jul 28 19:01:29 2022: ncks --mk_rec_dmn time files/test...
    NCO:          netCDF Operators version 5.1.0 (Homepage = http://nco.sf.ne...

## We would like to let trainee access to a zarr file we prepared.  
- creating a zarr file is a one line command, but it creates multiple files.
- How can we access to 'cli' interfafce to place files on swift, so that we can avoid uploading manually the file from web interface?




In [9]:
ds.to_zarr("./test.zarr" ,mode="w")
!ls -lart test.zarr/*

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/dataset.py:2036: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(


test.zarr/air:
total 28
-rw-r--r-- 1 jovyan jovyan  360 Jul 30 07:51 .zarray
-rw-r--r-- 1 jovyan jovyan  497 Jul 30 07:51 .zattrs
-rw-r--r-- 1 jovyan jovyan 8969 Jul 30 07:51 0.0.0
drwxr-xr-x 2 jovyan jovyan 4096 Jul 30 07:51 .
drwxr-xr-x 6 jovyan jovyan 4096 Jul 30 07:51 ..

test.zarr/lat:
total 20
-rw-r--r-- 1 jovyan jovyan  315 Jul 30 07:51 .zarray
-rw-r--r-- 1 jovyan jovyan  159 Jul 30 07:51 .zattrs
-rw-r--r-- 1 jovyan jovyan  116 Jul 30 07:51 0
drwxr-xr-x 2 jovyan jovyan 4096 Jul 30 07:51 .
drwxr-xr-x 6 jovyan jovyan 4096 Jul 30 07:51 ..

test.zarr/time:
total 20
-rw-r--r-- 1 jovyan jovyan  313 Jul 30 07:51 .zarray
-rw-r--r-- 1 jovyan jovyan  172 Jul 30 07:51 .zattrs
-rw-r--r-- 1 jovyan jovyan   32 Jul 30 07:51 0
drwxr-xr-x 2 jovyan jovyan 4096 Jul 30 07:51 .
drwxr-xr-x 6 jovyan jovyan 4096 Jul 30 07:51 ..

test.zarr/lon:
total 20
-rw-r--r-- 1 jovyan jovyan  315 Jul 30 07:51 .zarray
-rw-r--r-- 1 jovyan jovyan  160 Jul 30 07:51 .zattrs
-rw-r--r-- 1 jovyan jovyan  118 Jul 30 07:51 0